I think I'm running into a problem where prediction requests aren't getting routed to modelcontainers. This code can hopefully replicate that.

### Instantiate Clipper

In [ ]:
import sys
import numpy as np
import os
sys.path.append(os.path.abspath('../../management/'))
import clipper_manager
reload(clipper_manager)
# Change the username if necessary
user = ""
# Set the path to the SSH key
key = ""
# Set the SSH host
host = "localhost"
clipper = clipper_manager.Clipper(host, user, key)

clipper.start()

### Register app

In [ ]:
app_name = "noop_demo"
candidate_models = [
    {"model_name": "my_noop_model", "model_version": 1},
]

clipper.register_application(
    app_name,
    candidate_models,
    "ints",
    "EXP4",
    slo_micros=20000)

### Deploy Noop model-container

In [ ]:
path = "some/path/to/an/empty/directory"

# This step may take some time in downloading the container image
model_added = clipper.deploy_model(
    "my_noop_model",
    1,
    path,
    "nishadsingh/noop_container:latest",
    ["noop"],
    "ints",
    num_containers=1
)

print("Model deploy successful? {success}".format(success=model_added))

In [ ]:
clipper.get_app_info(app_name)

### See what it does when queried

In [ ]:
import json
import requests
def get_prediction(host, app, x):
    uid = 0
    url = "http://%s:1337/%s/predict" % (host, app)
    req_json = json.dumps({'uid': uid, 'input': list(x)})
    headers = {'Content-type': 'application/json'}
    r = requests.post(url, headers=headers, data=req_json)
    return r.text

In [ ]:
test_datapoint = np.asarray([1,2,3,4,5,6])
clipper_prediction = get_prediction(host, app_name, test_datapoint)
what_noop_would_have_returned = np.sum(test_datapoint)
print("Clipper's prediction: {cpred}, what the noop `predict_ints` would have returned: {npred}"\
      .format(cpred=clipper_prediction, npred=what_noop_would_have_returned))

### Check logs

Run `docker ps`, find the running instance of nishadsingh/noop_container, grab its container id and run `docker logs <container_id>`. If the Noop container's `predict_ints` (or any other `predict_<type>` for that matter) function was used, you'll see output.